# Analiza Olimpijskih Igra 2024

- Da li zemlje sa višim GDP-om osvajaju više medalja?
 - Da li su sportovi sa većim ulaganjima (skupi sportovi) koncentrisani u
ekonomski jacim zemljama (visok GDP po blavi stanovnika), dok su jeftini sportovi
uspešni u ekonomski slabijim zemljama?

- U skupe sportove spadaju: golf, jedrenje, konjički sportovi, tenis, biciklizam
- U jeftine sportove spadaju: atletika, boks, džudo, tekvondo i rvanje

## Prikupljanje podataka

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [2]:
pd.set_option("display.max_rows",None)

In [3]:
url = 'https://www.worldometers.info/gdp/gdp-by-country/'
request = get(url)

In [4]:
#request.text

In [5]:
soup_data = BeautifulSoup(request.text, 'html.parser')
country = []
gdp = []
population = []
gdp_per_capita = []
i = 1
for td in soup_data.find_all("td"):
    #print(td.text,i)
    if i%8 ==2:
        country.append(td.text)
    if i%8 ==3:
        gdp.append(td.text)
    if i%8 ==6:
        population.append(td.text)
    if i%8 ==7:
        gdp_per_capita.append(td.text)
    i+=1

In [6]:
country_table = pd.DataFrame(data = {'country':country,'gdp' : gdp,'population':population,'gdp_per_capita':gdp_per_capita})
country_table.head()

,country,gdp,population,gdp_per_capita
0,United States,"$25,462,700,000,000","341,534,046","$74,554"
1,China,"$17,963,200,000,000","1,425,179,569","$12,604"
2,Japan,"$4,231,140,000,000","124,997,578","$33,850"
3,Germany,"$4,072,190,000,000","84,086,227","$48,429"
4,India,"$3,385,090,000,000","1,425,423,212","$2,375"


In [7]:
country_table.shape

(177, 4)

In [8]:
medals = pd.read_csv('medals.csv')
medals.head()

,medal_type,medal_code,medal_date,name,gender,discipline,event,event_type,url_event,code,country_code,country,country_long
0,Gold Medal,1.0,2024-07-27,Remco EVENEPOEL,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903136,BEL,Belgium,Belgium
1,Silver Medal,2.0,2024-07-27,Filippo GANNA,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1923520,ITA,Italy,Italy
2,Bronze Medal,3.0,2024-07-27,Wout van AERT,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903147,BEL,Belgium,Belgium
3,Gold Medal,1.0,2024-07-27,Grace BROWN,W,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1940173,AUS,Australia,Australia
4,Silver Medal,2.0,2024-07-27,Anna HENDERSON,W,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1912525,GBR,Great Britain,Great Britain


In [9]:
medals.columns

Index(['medal_type', 'medal_code', 'medal_date', 'name', 'gender',
       'discipline', 'event', 'event_type', 'url_event', 'code',
       'country_code', 'country', 'country_long'],
      dtype='object')

## Čišćenje tabele medals

In [10]:
medals.isna().sum()

medal_type      0
medal_code      1
medal_date      0
name            0
gender          0
discipline      0
event           0
event_type      0
url_event       9
code            0
country_code    0
country         0
country_long    0
dtype: int64

In [11]:
medals[medals['medal_code'].isnull()]

,medal_type,medal_code,medal_date,name,gender,discipline,event,event_type,url_event,code,country_code,country,country_long
469,Bronze Medal,NaN,2024-08-04,Gregoria Mariska TUNJUNG,W,Badminton,Women's Singles,HATH,/en/paris-2024/results/badminton/women-s-singl...,1926315,INA,Indonesia,Indonesia


In [12]:
medals['medal_code'] = medals['medal_code'].fillna(3)

In [13]:
medals.isna().sum()

medal_type      0
medal_code      0
medal_date      0
name            0
gender          0
discipline      0
event           0
event_type      0
url_event       9
code            0
country_code    0
country         0
country_long    0
dtype: int64

In [14]:
medals[medals.duplicated(keep=False)]

,medal_type,medal_code,medal_date,name,gender,discipline,event,event_type,url_event,code,country_code,country,country_long


In [15]:
medals['medal_type'].value_counts()

Bronze Medal    385
Silver Medal    330
Gold Medal      329
Name: medal_type, dtype: int64

In [16]:
groups = medals.groupby(['discipline','event','medal_type']).size().reset_index(name = 'group_size')

In [17]:
groups[groups['group_size']>1]

,discipline,event,medal_type,group_size
27,Artistic Gymnastics,Men's Horizontal Bar,Bronze Medal,2
192,Athletics,Women's High Jump,Bronze Medal,2
240,Boxing,Men's +92kg,Bronze Medal,2
243,Boxing,Men's 51kg,Bronze Medal,2
246,Boxing,Men's 57kg,Bronze Medal,2
249,Boxing,Men's 63.5kg,Bronze Medal,2
252,Boxing,Men's 71kg,Bronze Medal,2
255,Boxing,Men's 80kg,Bronze Medal,2
258,Boxing,Men's 92kg,Bronze Medal,2
261,Boxing,Women's 50kg,Bronze Medal,2


In [18]:
medals['country'].unique().size

92

## Normalizacija kolone 'country'

In [19]:
medals['country'].unique()

array(['Belgium', 'Italy', 'Australia', 'Great Britain', 'United States',
       'China', 'Korea', 'Tunisia', 'Hong Kong, China', 'France',
       'Hungary', 'Kazakhstan', 'Japan', 'Spain', 'Mongolia', 'Sweden',
       'Fiji', 'South Africa', 'Germany', 'Canada', 'Mexico', 'Poland',
       'Egypt', 'Brazil', 'Republic of Moldova', 'Uzbekistan', 'Kosovo',
       'India', 'Türkiye', 'Slovakia', 'Ukraine', 'Azerbaijan', 'Croatia',
       'Switzerland', 'Romania', 'Ireland', 'Georgia', 'Tajikistan',
       'Slovenia', 'New Zealand', 'Serbia', 'Guatemala', 'DPR Korea',
       'Argentina', 'Greece', 'Austria', 'Netherlands', 'Ecuador',
       'Israel', 'Portugal', 'Uganda', 'Ethiopia', 'Czechia', 'AIN',
       'Philippines', 'Norway', 'Grenada', 'Jamaica', 'Saint Lucia',
       'Dominica', 'Chinese Taipei', 'Denmark', 'Lithuania', 'Armenia',
       'Algeria', 'Malaysia', 'Indonesia', 'Dominican Republic',
       'Cabo Verde', 'Cuba', 'Chile', 'Colombia', 'Kenya', 'Thailand',
       'Bahrain'

In [20]:
country_table['country'].unique()

array(['United States', 'China', 'Japan', 'Germany', 'India',
       'United Kingdom', 'France', 'Russia', 'Canada', 'Italy', 'Brazil',
       'Australia', 'South Korea', 'Mexico', 'Spain', 'Indonesia',
       'Saudi Arabia', 'Netherlands', 'Turkey', 'Switzerland', 'Poland',
       'Argentina', 'Sweden', 'Norway', 'Belgium', 'Ireland', 'Israel',
       'United Arab Emirates', 'Thailand', 'Nigeria', 'Egypt', 'Austria',
       'Singapore', 'Bangladesh', 'Vietnam', 'Malaysia', 'South Africa',
       'Philippines', 'Denmark', 'Iran', 'Pakistan', 'Hong Kong',
       'Colombia', 'Romania', 'Chile', 'Czech Republic (Czechia)',
       'Finland', 'Iraq', 'Portugal', 'New Zealand', 'Peru', 'Qatar',
       'Kazakhstan', 'Greece', 'Algeria', 'Kuwait', 'Hungary', 'Ukraine',
       'Morocco', 'Ethiopia', 'Slovakia', 'Ecuador', 'Oman',
       'Dominican Republic', 'Kenya', 'Angola', 'Guatemala', 'Bulgaria',
       'Luxembourg', 'Uzbekistan', 'Azerbaijan', 'Panama', 'Tanzania',
       'Sri Lanka', 'Gh

In [21]:
for cnt in medals['country'].unique():
    if cnt not in country_table['country'].unique():
        print(cnt)

Great Britain
Korea
Hong Kong, China
Republic of Moldova
Kosovo
Türkiye
DPR Korea
Czechia
AIN
Chinese Taipei
Cuba
IR Iran
EOR
Puerto Rico


In [22]:
replace_values = {"Great Britain": "United Kingdom", "Korea": "South Korea", "Republic of Moldova": "Moldova", 
                  "DPR Korea": "North Korea","IR Iran":"Iran","Türkiye": "Turkey",}
medals = medals.replace({'country':replace_values})

In [23]:
country_table['country'] = country_table['country'].replace('Czech Republic (Czechia)','Czechia')

In [24]:
country_table.loc[len(country_table.index)] = ['Cuba','$107,352,000,000','11,176,354','$7,252']

In [25]:
to_delete = []
for cnt in medals['country'].unique():
    if cnt not in country_table['country'].unique():
        to_delete.append(cnt)

In [26]:
medals = medals[~medals.country.isin(to_delete)]

## Čišćenje tabele country

In [27]:
country_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 177
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   country         178 non-null    object
 1   gdp             178 non-null    object
 2   population      178 non-null    object
 3   gdp_per_capita  178 non-null    object
dtypes: object(4)
memory usage: 7.0+ KB


In [28]:
country_table.head()

,country,gdp,population,gdp_per_capita
0,United States,"$25,462,700,000,000","341,534,046","$74,554"
1,China,"$17,963,200,000,000","1,425,179,569","$12,604"
2,Japan,"$4,231,140,000,000","124,997,578","$33,850"
3,Germany,"$4,072,190,000,000","84,086,227","$48,429"
4,India,"$3,385,090,000,000","1,425,423,212","$2,375"


In [29]:
country_table.gdp = country_table.gdp.str.replace('$','')
country_table.gdp = country_table.gdp.str.replace(',','')

C:\Users\m.slijepcevic\AppData\Local\Temp\ipykernel_15916\3007408281.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  country_table.gdp = country_table.gdp.str.replace('$','')


In [30]:
country_table.gdp_per_capita = country_table.gdp_per_capita.str.replace('$','')
country_table.gdp_per_capita = country_table.gdp_per_capita.str.replace(',','')

C:\Users\m.slijepcevic\AppData\Local\Temp\ipykernel_15916\1854651175.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  country_table.gdp_per_capita = country_table.gdp_per_capita.str.replace('$','')


In [31]:
country_table.population = country_table.population.str.replace(',','')

In [32]:
country_table.gdp = country_table.gdp.astype(float)
country_table.gdp_per_capita = country_table.gdp_per_capita.astype(float)
country_table.population = country_table.population.astype(float)

In [33]:
country_table.head()

,country,gdp,population,gdp_per_capita
0,United States,2.546270e+13,3.415340e+08,74554.0
1,China,1.796320e+13,1.425180e+09,12604.0
2,Japan,4.231140e+12,1.249976e+08,33850.0
3,Germany,4.072190e+12,8.408623e+07,48429.0
4,India,3.385090e+12,1.425423e+09,2375.0


In [34]:
country_table.to_csv('country_for_analysis.csv')

In [35]:
medals.to_csv('medals_for_analysis.csv')

## Analiza podataka

### GDP i broj medalja

In [36]:
medals_grouped = medals.groupby(['country']).size().reset_index(name = 'count')

In [37]:
medals_grouped.head()

,country,count
0,Albania,2
1,Algeria,3
2,Argentina,3
3,Armenia,4
4,Australia,53


In [38]:
medals_grouped

,country,count
0,Albania,2
1,Algeria,3
2,Argentina,3
3,Armenia,4
4,Australia,53
5,Austria,5
6,Azerbaijan,7
7,Bahrain,4
8,Belgium,10
9,Botswana,2


In [39]:
merged = medals_grouped.merge(country_table, how = 'inner', on = 'country')
merged.head()

,country,count,gdp,population,gdp_per_capita
0,Albania,2,1.888210e+10,2827608.0,6678.0
1,Algeria,3,1.919130e+11,45477389.0,4220.0
2,Argentina,3,6.327700e+11,45407904.0,13935.0
3,Armenia,4,1.950278e+10,2880874.0,6770.0
4,Australia,53,1.675420e+12,26200984.0,63945.0


In [40]:
numeric = merged.select_dtypes(include = 'number')
numeric.corr()

,count,gdp,population,gdp_per_capita
count,1.000000,0.858858,0.392787,0.341684
gdp,0.858858,1.000000,0.570042,0.229968
population,0.392787,0.570042,1.000000,-0.102252
gdp_per_capita,0.341684,0.229968,-0.102252,1.000000


In [41]:
merged['medals per gdp'] = round(merged['count']*100000000000/merged['gdp'])
merged.sort_values(by = 'medals per gdp',ascending = False).reset_index()

,index,country,count,gdp,population,gdp_per_capita,medals per gdp
0,23,Dominica,1,6.120481e+08,6.682600e+04,9159.0,163.0
1,33,Grenada,2,1.256413e+09,1.169130e+05,10747.0,159.0
2,65,Saint Lucia,2,2.065028e+09,1.787810e+05,11551.0,97.0
3,47,Kyrgyzstan,6,1.093064e+10,6.955788e+06,1571.0,55.0
4,12,Cabo Verde,1,2.314817e+09,5.197410e+05,4454.0,43.0
5,42,Jamaica,6,1.709776e+10,2.839144e+06,6022.0,35.0
6,75,Tajikistan,3,1.049212e+10,1.018222e+07,1030.0,29.0
7,51,Moldova,4,1.442095e+10,3.039985e+06,4744.0,28.0
8,30,Georgia,7,2.460538e+10,3.794784e+06,6484.0,28.0
9,3,Armenia,4,1.950278e+10,2.880874e+06,6770.0,21.0


In [42]:
merged.describe()

,count,gdp,population,gdp_per_capita,medals per gdp
count,85.000000,8.500000e+01,8.500000e+01,85.000000,85.000000
mean,11.964706,1.068165e+12,7.250383e+07,22539.847059,10.741176
std,20.272810,3.395509e+12,2.199779e+08,25479.218301,27.256783
min,1.000000,6.120481e+08,6.682600e+04,963.000000,0.000000
25%,2.000000,6.350175e+10,5.456801e+06,4965.000000,1.000000
50%,5.000000,2.372960e+11,1.211933e+07,10996.000000,2.000000
75%,10.000000,5.792670e+11,5.173794e+07,29367.000000,8.000000
max,126.000000,2.546270e+13,1.425423e+09,106155.000000,163.000000


In [43]:
gdp_bins = [0,60000000000,200000000000,600000000000, float('inf')]
gdp_labels = ['Low GDP','Lower Middle GDP','Upper Middle GDP','High GDP']

merged['GDP category'] = pd.cut(merged['gdp'], bins = gdp_bins, labels = gdp_labels)
merged.head()

,country,count,gdp,population,gdp_per_capita,medals per gdp,GDP category
0,Albania,2,1.888210e+10,2827608.0,6678.0,11.0,Low GDP
1,Algeria,3,1.919130e+11,45477389.0,4220.0,2.0,Lower Middle GDP
2,Argentina,3,6.327700e+11,45407904.0,13935.0,0.0,High GDP
3,Armenia,4,1.950278e+10,2880874.0,6770.0,21.0,Low GDP
4,Australia,53,1.675420e+12,26200984.0,63945.0,3.0,High GDP


In [44]:
gdp_medal_summary = merged.groupby('GDP category')['count'].sum().reset_index()
gdp_medal_summary

,GDP category,count
0,Low GDP,54
1,Lower Middle GDP,119
2,Upper Middle GDP,153
3,High GDP,691


In [45]:
medals.head()

,medal_type,medal_code,medal_date,name,gender,discipline,event,event_type,url_event,code,country_code,country,country_long
0,Gold Medal,1.0,2024-07-27,Remco EVENEPOEL,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903136,BEL,Belgium,Belgium
1,Silver Medal,2.0,2024-07-27,Filippo GANNA,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1923520,ITA,Italy,Italy
2,Bronze Medal,3.0,2024-07-27,Wout van AERT,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903147,BEL,Belgium,Belgium
3,Gold Medal,1.0,2024-07-27,Grace BROWN,W,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1940173,AUS,Australia,Australia
4,Silver Medal,2.0,2024-07-27,Anna HENDERSON,W,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1912525,GBR,United Kingdom,Great Britain


### Skupi vs Jefrini spotovi

### Prosečan GDP po glavi stanovnika po sportu

In [47]:
medals_grouped = medals.groupby(['discipline','country']).size().reset_index(name = 'medal_count')
medals_grouped.head()

,discipline,country,medal_count
0,3x3 Basketball,France,1
1,3x3 Basketball,Germany,1
2,3x3 Basketball,Lithuania,1
3,3x3 Basketball,Netherlands,1
4,3x3 Basketball,Spain,1


In [48]:
merged_data = medals_grouped.merge(country_table, on='country', how = 'inner')
merged_data.head()

,discipline,country,medal_count,gdp,population,gdp_per_capita
0,3x3 Basketball,France,1,2.782910e+12,66277409.0,41989.0
1,Archery,France,2,2.782910e+12,66277409.0,41989.0
2,Athletics,France,1,2.782910e+12,66277409.0,41989.0
3,Basketball,France,2,2.782910e+12,66277409.0,41989.0
4,Boxing,France,3,2.782910e+12,66277409.0,41989.0


In [60]:
sport_summary = merged_data.groupby('discipline').agg(total_medals = ('medal_count','sum'), avg_gdp_per_capita = ('gdp_per_capita','mean')).reset_index()

In [61]:
sport_summary.head()

,discipline,total_medals,avg_gdp_per_capita
0,3x3 Basketball,6,45752.666667
1,Archery,15,33019.714286
2,Artistic Gymnastics,42,27667.312500
3,Artistic Swimming,6,43354.200000
4,Athletics,144,26569.976190


In [62]:
sorted_sports = sport_summary.sort_values(by = 'avg_gdp_per_capita', ascending = False)


    U skupe sportove spadaju: golf, jedrenje, konjički sportovi, tenis, biciklizam
    U jeftine sportove spadaju: atletika, boks, džudo, tekvondo i rvanje



In [63]:
sorted_sports

,discipline,total_medals,avg_gdp_per_capita
13,Cycling BMX Racing,6,63289.000000
7,Beach Volleyball,6,61095.166667
22,Handball,6,58555.400000
40,Triathlon,9,58342.166667
18,Equestrian,18,58001.333333
15,Cycling Road,12,52043.285714
30,Sailing,30,48354.736842
28,Rowing,41,47839.533333
16,Cycling Track,36,47500.666667
6,Basketball,6,47459.750000
